In [11]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

In [12]:
train_transform = transforms.Compose([
        transforms.RandomRotation(10),     
        transforms.RandomHorizontalFlip(),  
        transforms.Resize(224),             
        transforms.CenterCrop(224),         
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_data = datasets.ImageFolder(root="../datasets/cats-and-dogs/train", transform=train_transform)
test_data = datasets.ImageFolder(root="../datasets/cats-and-dogs/test", transform=test_transform)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8)

In [17]:
for i, label in enumerate(train_data.classes):
    print(f"Label: {label}, index: {i}")

Label: cat, index: 0
Label: dog, index: 1


In [18]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16 * 54 * 54, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, 2)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16 * 54 * 54)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        
        return F.log_softmax(X, dim=1)
    

In [19]:
cnn_model = ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

In [20]:
epochs = 5
for epoch in range(epochs):
    epoch += 1

    trn_corr = 0
    for batch, (X_train, y_train) in enumerate(train_loader):
        y_pred = cnn_model(X_train)
        loss = criterion(y_pred, y_train)

        # How many predicted correct.
        trn_corr += (torch.max(y_pred.data, 1)[1] == y_train).sum()
        
        batch += 1
        if batch % 200 == 0:
            print(f"TRAIN: epoch: {epoch} loss: {loss.item()} batch: {batch} accuracy: {trn_corr.item() * 100 / (8 * batch):7.3f}%")

        # Train.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        tst_corr = 0
        for batch, (X_test, y_test) in enumerate(test_loader):
            y_pred = cnn_model(X_test)
            loss = criterion(y_pred, y_test)
            # How many predicted correct.
            tst_corr += (torch.max(y_pred.data,1)[1] == y_test).sum()

            batch += 1
            if batch % 50 == 0:
                print(f"TEST: epoch: {epoch} loss: {loss.item()} batch: {batch} accuracy: {tst_corr.item() * 100 / (8 * batch):7.3f}%")

TRAIN: epoch: 1 loss: 0.5147902965545654 batch: 200 accuracy:  56.938%
TRAIN: epoch: 1 loss: 0.6590794324874878 batch: 400 accuracy:  59.438%
TRAIN: epoch: 1 loss: 0.4818272590637207 batch: 600 accuracy:  60.729%
TRAIN: epoch: 1 loss: 0.6689613461494446 batch: 800 accuracy:  61.953%
TRAIN: epoch: 1 loss: 0.531286895275116 batch: 1000 accuracy:  62.688%
TRAIN: epoch: 1 loss: 0.59315025806427 batch: 1200 accuracy:  63.625%
TRAIN: epoch: 1 loss: 0.43596112728118896 batch: 1400 accuracy:  64.696%
TRAIN: epoch: 1 loss: 0.7025898098945618 batch: 1600 accuracy:  65.359%
TRAIN: epoch: 1 loss: 0.43715700507164 batch: 1800 accuracy:  66.056%
TRAIN: epoch: 1 loss: 0.7208001613616943 batch: 2000 accuracy:  66.675%
TRAIN: epoch: 1 loss: 0.5569039583206177 batch: 2200 accuracy:  67.188%
TRAIN: epoch: 1 loss: 0.46903637051582336 batch: 2400 accuracy:  67.651%
TRAIN: epoch: 1 loss: 0.4140457212924957 batch: 2600 accuracy:  68.149%
TRAIN: epoch: 1 loss: 0.4694763422012329 batch: 2800 accuracy:  68.522%

In [21]:
torch.save(cnn_model.state_dict(), "../../models/cats-and-dogs.pt")